# Assignment

In [57]:
# Import 
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from Chess_env import *


random.seed(1)
np.random.seed(1)
size_board = 4

## The Environment

You can find the environment in the file Chess_env, which contains the class Chess_env. To define an object, you need to provide the board size considered as input. In our example, size_board=4. 
Chess_env is composed by the following methods:

1. Initialise_game. The method initialises an episode by placing the three pieces considered (Agent's king and queen, enemy's king) in the chess board. The outputs of the method are described below in order.

     S $\;$ A matrix representing the board locations filled with 4 numbers: 0, no piece in that position; 1, location of the 
     agent's king; 2 location of the queen; 3 location of the enemy king.
     
     X $\;$ The features, that is the input to the neural network. See the assignment for more information regarding the            definition of the features adopted. To personalise this, go into the Features method of the class Chess_env() and change        accordingly.
     
     allowed_a $\;$ The allowed actions that the agent can make. The agent is moving a king, with a total number of 8                possible actions, and a queen, with a total number of $(board_{size}-1)\times 8$ actions. The total number of possible actions correspond      to the sum of the two, but not all actions are allowed in a given position (movements to locations outside the borders or      against chess rules). Thus, the variable allowed_a is a vector that is one (zero) for an action that the agent can (can't)      make. Be careful, apply the policy considered on the actions that are allowed only.
     

2. OneStep. The method performs a one step update of the system. Given as input the action selected by the agent, it updates the chess board by performing that action and the response of the enemy king (which is a random allowed action in the settings considered). The first three outputs are the same as for the Initialise_game method, but the variables are computed for the position reached after the update of the system. The fourth and fifth outputs are:

     R $\;$ The reward. To change this, look at the OneStep method of the class where the rewards are set.
     
     Done $\;$ A variable that is 1 if the episode has ended (checkmate or draw).
     
     
3. Features. Given the chessboard position, the method computes the features.

This information and a quick analysis of the class should be all you need to get going. The other functions that the class exploits are uncommented and constitute an example on how not to write a python code. You can take a look at them if you want, but it is not necessary.






In [58]:
## INITIALISE THE ENVIRONMENT
env = Chess_Env(size_board)

In [59]:
## PRINT 5 STEPS OF AN EPISODE CONSIDERING A RANDOM AGENT

S, X, allowed_a = env.Initialise_game()  # INTIALISE GAME

print(S)  # PRINT CHESS BOARD (SEE THE DESCRIPTION ABOVE)

print('check? ', env.check)  # PRINT VARIABLE THAT TELLS IF ENEMY KING IS IN CHECK (1) OR NOT (0)
print('dofk2 ', np.sum(env.dfk2_constrain).astype(int))  # PRINT THE NUMBER OF LOCATIONS THAT THE ENEMY KING CAN MOVE TO

for i in range(5):

    a, _ = np.where(allowed_a == 1)  # FIND WHAT THE ALLOWED ACTIONS ARE
    a_agent = np.random.permutation(a)[0]  # MAKE A RANDOM ACTION

    S, X, allowed_a, R, Done = env.OneStep(a_agent)  # UPDATE THE ENVIRONMENT

    ## PRINT CHESS BOARD AND VARIABLES
    print('')
    print(S)
    print(R, '', Done)
    print('check? ', env.check)
    print('dofk2 ', np.sum(env.dfk2_constrain).astype(int))

    # TERMINATE THE EPISODE IF Done=True (DRAW OR CHECKMATE)
    if Done:
        break



[[0 0 0 0]
 [0 1 0 0]
 [0 0 0 2]
 [3 0 0 0]]
check?  0
dofk2  1

[[0 0 1 0]
 [0 0 0 0]
 [0 0 0 2]
 [0 3 0 0]]
0  0
check?  0
dofk2  1

[[0 0 0 0]
 [0 0 0 1]
 [0 0 0 2]
 [3 0 0 0]]
0  0
check?  0
dofk2  1

[[0 0 0 0]
 [0 0 2 1]
 [3 0 0 0]
 [0 0 0 0]]
0  0
check?  0
dofk2  1

[[0 0 0 1]
 [0 0 2 0]
 [0 0 0 0]
 [0 3 0 0]]
0  0
check?  0
dofk2  1

[[0 0 0 0]
 [0 0 2 1]
 [3 0 0 0]
 [0 0 0 0]]
0  0
check?  0
dofk2  1


In [60]:
# PERFORM N_episodes=1000 EPISODES MAKING RANDOM ACTIONS AND COMPUTE THE AVERAGE REWARD AND NUMBER OF MOVES 

S, X, allowed_a = env.Initialise_game()
N_episodes = 1000

# VARIABLES WHERE TO SAVE THE FINAL REWARD IN AN EPISODE AND THE NUMBER OF MOVES 
R_save_random = np.zeros([N_episodes, 1])
N_moves_save_random = np.zeros([N_episodes, 1])

for n in range(N_episodes):

    S, X, allowed_a = env.Initialise_game()  # INITIALISE GAME
    Done = 0  # SET Done=0 AT THE BEGINNING
    i = 1  # COUNTER FOR THE NUMBER OF ACTIONS (MOVES) IN AN EPISODE

    # UNTIL THE EPISODE IS NOT OVER...(Done=0)
    while Done == 0:

        # SAME AS THE CELL BEFORE, BUT SAVING THE RESULTS WHEN THE EPISODE TERMINATES 

        a, _ = np.where(allowed_a == 1)
        a_agent = np.random.permutation(a)[0]

        S, X, allowed_a, R, Done = env.OneStep(a_agent)

        if Done:
            R_save_random[n] = np.copy(R)
            N_moves_save_random[n] = np.copy(i)
            break

        i = i + 1  # UPDATE THE COUNTER

# AS YOU SEE, THE PERFORMANCE OF A RANDOM AGENT ARE NOT GREAT, SINCE THE MAJORITY OF THE POSITIONS END WITH A DRAW
# (THE ENEMY KING IS NOT IN CHECK AND CAN'T MOVE)

print('Random_Agent, Average reward:', np.mean(R_save_random), 'Number of steps: ', np.mean(N_moves_save_random))

Random_Agent, Average reward: 0.199 Number of steps:  7.282


## Task 3
The following is an implementation of a general **Neural Network Class** and a reinforcement learner using **SARSA**.

In [61]:
class Network:
    W1, W2 = None, None

    def __init__(self, K, input_dim, output_dim, eta=0.02, rho=0.9, rmsprop=False):
        # Xavier initialization
        self.W1 = np.random.randn(K + 1, input_dim + 1) * 1.0 / np.sqrt(input_dim + 1)
        self.W2 = np.random.randn(output_dim, K + 1) * 1.0 / np.sqrt(K + 1)
        # Step size
        self.eta = eta
        # RMSprop parameters
        self.rho = rho
        self.V1 = np.zeros(self.W1.shape)
        self.V2 = np.zeros(self.W2.shape)
        self.l1 = None
        self.l2 = None
        self.rmsprop = rmsprop
       
    @staticmethod
    def relu(A):
        return np.maximum(0, A)

    @staticmethod
    def gradient(X, T, Y, H, W2, Z1, Z2):
        # Add bias term
        X_bias = np.vstack((np.ones(X.shape[1]), X))

        # d Loss / d Y
        G_Y = 2. * (Y - T)
        # d Y / d Z2
        G_Z2 = G_Y * np.maximum(0, np.sign(Z2))
        # d Z2 / d W2
        G_W2 = np.dot(G_Z2, H.T)
        # Layer 2 gradient
        G2 = (1. / X_bias.shape[1]) * G_W2

        # d Z2 / d H
        G_H = np.dot(W2.T, G_Z2)
        # d H / d Z1
        G_Z1 = G_H * np.maximum(0, np.sign(Z1))
        # d Z1 / d W1
        G_W1 = np.dot(G_Z1, X_bias.T)
        # Layer 1 gradient
        G1 = (1. / X_bias.shape[1]) * G_W1
        return G1, G2

    def descent(self, X, T, H, Y, Z1, Z2):
        G1, G2 = Network.gradient(X, T, Y, H, self.W2, Z1, Z2)

        if self.rmsprop:
            self.V1 = self.rho * self.V1 + (1 - self.rho) * np.square(G1)
            self.V2 = self.rho * self.V2 + (1 - self.rho) * np.square(G2)

            self.l1 = (self.eta / (np.sqrt(self.V1) + 1e-7))
            self.l2 = (self.eta / (np.sqrt(self.V2) + 1e-7))

            self.W1 -= self.l1 * G1
            self.W2 -= self.l2 * G2
        else:
            self.W1 -= self.eta * G1
            self.W2 -= self.eta * G2

    def forward(self, X):
        # Add Bias to first layer input
        X_bias = np.vstack((np.ones(X.shape[1]), X))
        # First Layer
        Z1 = np.dot(self.W1, X_bias)
        H = Network.relu(Z1)
        # Fix Bias for second layer input
        H[0, :] = 1.
        # Second Layer
        Z2 = np.dot(self.W2, H)
        Y = Network.relu(Z2)
        return Y, H, Z1, Z2

In [62]:
def epsilon_greedy_policy(Qvalues, epsilon):
    N_class = np.shape(Qvalues)[0]
    batch_size = np.shape(Qvalues)[1]

    rand_values = np.random.uniform(0, 1, [batch_size])

    rand_a = rand_values < epsilon
    a = np.zeros([batch_size, N_class])

    for i in range(batch_size):
        if rand_a[i]:
            valid_moves = np.where(Qvalues[:,i] > -1000)[0]
            chosen = np.random.choice(valid_moves)
            a[i, chosen] = 1
        else:
            a[i, np.argmax(Qvalues[:, i])] = 1

    return a

In [63]:
def test(network, nr_episodes=10000):
    nr_moves = []
    total_rewards = []
    nr_stuck = 0
    for i in range(nr_episodes):
	    done = 0
	    _, X, allowed_a = env.Initialise_game()
	    X = X.reshape(len(X), 1)
	    total_reward = 0
	    count = 0
	    while done == 0:
		    count += 1
		    Q_values, _, _, _ = network.forward(X)
		    masked_Q_values = Q_values - (1 - allowed_a) * 100000
		    a_agent = epsilon_greedy_policy(masked_Q_values, 0).T
		    _, X, _, R, done = env.OneStep(np.argmax(a_agent))
		    X = np.array(X).reshape(len(X),1)
		    total_reward += R
            if count > 100:
                nr_stuck += 1
                break
	    nr_moves.append(count)
	    total_rewards.append(total_reward)
    return nr_moves, total_rewards, nr_stuck

In [64]:
def plot(moves, rewards, name):
    ema_rewards = pd.DataFrame(rewards).ewm(halflife=1000).mean()[100:]
    plt.figure()
    plt.title(f"{name} Reward / Game")
    plt.plot(ema_rewards)
    plt.show()

    ema_moves = pd.DataFrame(moves).ewm(halflife=1000).mean()[100:]
    plt.figure()
    plt.title(f"{name} Moves / Game")
    plt.plot(ema_moves)
    plt.show()

In [65]:
def train_and_test(algo, network, name):
    moves, rewards = algo(network)
    print(f'\n{name} Agent training, Average reward: {np.mean(rewards)}, Number of steps: {np.mean(moves)}')
    moves_test, rewards_test, stuck = test(network)
    print(f'\n{name} Agent testing, Average reward: {np.mean(rewards_test)}, Number of steps: {np.mean(moves_test)}, Stuck: {stuck}')
    plot(moves, rewards, name)

In [66]:
def sarsa(network):
    count = []
    rewards = []

    for n in range(N_episodes):

        epsilon_f = epsilon_0 / (1 + beta * n)  ## DECAYING EPSILON
        Done = 0  ## SET DONE TO ZERO (BEGINNING OF THE EPISODE)
        i = 1  ## COUNTER FOR NUMBER OF ACTIONS
        total_reward = 0 ## COUNTER FOR TOTAL REWARD

        S, X, allowed_a = env.Initialise_game()  ## INITIALISE GAME
        X = X.reshape(len(X), 1)

        if n > 0 and n % 100 == 0:
            print(f"\rEp.: {n}, epsilon: {epsilon_f:.3f}, moves: {np.mean(count[n - 100:]):.2f}", end="")

        Q_values, H, Z1, Z2 = network.forward(X)
        masked_Q_values = Q_values - (1 - allowed_a) * 100000
        a_agent = epsilon_greedy_policy(masked_Q_values, epsilon_f).T

        while Done == 0:  ## START THE EPISODE

            S_next, X_next, allowed_a_next, R, Done = env.OneStep(np.argmax(a_agent))
            X_next = np.array(X_next).reshape(len(X_next), 1)
            
            total_reward += R

            ## THE EPISODE HAS ENDED, UPDATE...BE CAREFUL, THIS IS THE LAST STEP OF THE EPISODE
            if Done == 1:
                output = Q_values * a_agent
                target = R * a_agent
                network.descent(X, target, H, output, Z1, Z2)
                count.append(i)
                rewards.append(total_reward)
                break

            # IF THE EPISODE IS NOT OVER...
            else:
                Q_values_next, H_next, Z1_next, Z2_next = network.forward(X_next)
                masked_Q_values_next = Q_values_next - (1 - allowed_a_next) * 100000
                a_agent_next = epsilon_greedy_policy(masked_Q_values_next, epsilon_f).T
                future_R = Q_values_next[np.argmax(a_agent_next)]
                output = Q_values * a_agent
                target = (R + gamma * future_R) * a_agent
                network.descent(X, target, H, output, Z1, Z2)

            # NEXT STATE AND CO. BECOME ACTUAL STATE...     
            S = np.copy(S_next)
            X = np.copy(X_next)
            allowed_a = np.copy(allowed_a_next)
            Q_values = np.copy(Q_values_next)
            H = np.copy(H_next)
            a_agent = np.copy(a_agent_next)
            Z1 = np.copy(Z1_next)
            Z2 = np.copy(Z2_next)

            i += 1  # UPDATE COUNTER FOR NUMBER OF ACTIONS
    return count, rewards

In [67]:
# INITIALISE THE PARAMETERS OF YOUR NEURAL NETWORK AND...
# PLEASE CONSIDER TO USE A MASK OF ONE FOR THE ACTION MADE AND ZERO OTHERWISE IF YOU ARE NOT USING VANILLA GRADIENT DESCENT...
# WE SUGGEST A NETWORK WITH ONE HIDDEN LAYER WITH SIZE 200.


S, X, allowed_a = env.Initialise_game()
N_a = np.shape(allowed_a)[0]  # TOTAL NUMBER OF POSSIBLE ACTIONS

N_in = np.shape(X)[0]  ## INPUT SIZE
N_h = 200  ## NUMBER OF HIDDEN NODES

# HYPERPARAMETERS SUGGESTED (FOR A GRID SIZE OF 4)

epsilon_0 = 0.2  # STARTING VALUE OF EPSILON FOR THE EPSILON-GREEDY POLICY
beta = 0.00005  # THE PARAMETER SETS HOW QUICKLY THE VALUE OF EPSILON IS DECAYING (SEE epsilon_f BELOW)
gamma = 0.85  # THE DISCOUNT FACTOR
eta = 0.0035  # THE LEARNING RATE

N_episodes = 100000  # THE NUMBER OF GAMES TO BE PLAYED

## INITALISE YOUR NEURAL NETWORK...
network_sarsa = Network(N_h, N_in, N_a, eta=eta)

In [68]:
train_and_test(sarsa, network_sarsa, "SARSA")

Ep.: 99900, epsilon: 0.033, moves: 2.08
SARSA Agent training, Average reward: 0.94284 Number of steps: 2.66473


KeyboardInterrupt: 

## Task 4
We decreased **gamma to 0.7** and decreased **beta to 0.0001**.

In [ ]:
beta = 0.0001
gamma = 0.7     

## INITALISE YOUR NEURAL NETWORK...
network_adapted_sarsa = Network(N_h, N_in, N_a, eta=eta)

In [ ]:
train_and_test(sarsa, network_adapted_sarsa, "SARSA Adapted")

## Task 5
We implemented **Q-Learning** and **Experience Replay** as well.

### Q-Learning

In [ ]:
def qlearn(network):
    count = []
    rewards = []
    
    for n in range(N_episodes):

        epsilon_f = epsilon_0 / (1 + beta * n)  ## DECAYING EPSILON
        Done = 0  ## SET DONE TO ZERO (BEGINNING OF THE EPISODE)
        i = 1  ## COUNTER FOR NUMBER OF ACTIONS
        total_reward = 0 ## COUNTER FOR TOTAL REWARD

        S, X, allowed_a = env.Initialise_game()  ## INITIALISE GAME
        X = X.reshape(len(X), 1)

        if n > 0 and n % 100 == 0:
            print(f"\rEp.: {n}, epsilon: {epsilon_f:.3f}, moves: {np.mean(count[n - 100:]):.2f}", end="")

        Q_values, H, Z1, Z2 = network.forward(X)
        masked_Q_values = Q_values - (1 - allowed_a) * 100_000
        a_agent = epsilon_greedy_policy(masked_Q_values, epsilon_f).T

        while Done == 0:  ## START THE EPISODE

            S_next, X_next, allowed_a_next, R, Done = env.OneStep(np.argmax(a_agent))
            X_next = np.array(X_next).reshape(len(X_next), 1)
            total_reward += R

            ## THE EPISODE HAS ENDED, UPDATE...BE CAREFUL, THIS IS THE LAST STEP OF THE EPISODE
            if Done == 1:
                output = Q_values * a_agent
                target = R * a_agent
                network.descent(X, target, H, output, Z1, Z2)
                count.append(i)
                rewards.append(total_reward)
                break

            # IF THE EPISODE IS NOT OVER...
            else:
                Q_values_next, H_next, Z1_next, Z2_next = network.forward(X_next)
                masked_Q_values_next = Q_values_next - (1 - allowed_a_next) * 100000
                # Q-Learning chooses the next step greedily (epsilon=0)
                a_agent_next = epsilon_greedy_policy(masked_Q_values_next, 0.0).T
                future_R = Q_values_next[np.argmax(a_agent_next)]
                output = Q_values * a_agent
                target = (R + gamma * future_R) * a_agent
                network.descent(X, target, H, output, Z1, Z2)

            # NEXT STATE AND CO. BECOME ACTUAL STATE...     
            S = np.copy(S_next)
            X = np.copy(X_next)
            allowed_a = np.copy(allowed_a_next)
            Q_values = np.copy(Q_values_next)
            H = np.copy(H_next)
            Z1 = np.copy(Z1_next)
            Z2 = np.copy(Z2_next)
            # Q-Learning chooses next action based on greedy policty
            a_agent = epsilon_greedy_policy(masked_Q_values_next, epsilon_f).T

            i += 1  # UPDATE COUNTER FOR NUMBER OF ACTIONS
    return count, rewards

In [ ]:
## INITALISE YOUR NEURAL NETWORK...
network_qlearn = Network(N_h, N_in, N_a, eta=eta)

In [ ]:
train_and_test(qlearn, network_qlearn, "Q-Learning")

### Experience Replay

In [ ]:
class Experience:
    def __init__(self, state_before, state_after, action, reward):
        self.state_before = state_before
        self.state_after = state_after
        self.action = action
        self.reward = reward

In [ ]:
def replay(network):
    count = []
    rewards = []
    
    experiences = []
    batch_size = 50
    relevant_histories = 300
    
    for n in range(int(N_episodes/5)):

        epsilon_f = epsilon_0 / (1 + beta * n)  ## DECAYING EPSILON

        Done = 0  ## SET DONE TO ZERO (BEGINNING OF THE EPISODE)
        i = 1  ## COUNTER FOR NUMBER OF ACTIONS
        total_reward = 0 ## COUNTER FOR TOTAL REWARD

        S, X, allowed_a = env.Initialise_game()  ## INITIALISE GAME
        X = X.reshape(len(X), 1)

        if n > 0 and n % 100 == 0:
            print(f"\rEp.: {n}, epsilon: {epsilon_f:.3f}, moves: {np.mean(count[n - 100:]):.2f}", end="")

        while Done == 0:  ## START THE EPISODE
            
            Q_values, H, Z1, Z2 = network.forward(X)
            masked_Q_values = Q_values - (1 - allowed_a) * 100_000
            a_agent = epsilon_greedy_policy(masked_Q_values, epsilon_f).T

            S_next, X_next, allowed_a_next, R, Done = env.OneStep(np.argmax(a_agent))
            X_next = np.array(X_next).reshape(len(X_next), 1)
            total_reward += R
            
            if X_next.shape[0] == 0:
                expi = Experience(X, X, a_agent, R)
            else:
                expi = Experience(X, X_next, a_agent, R)
            experiences.append(expi)
            
            if len(experiences) > relevant_histories:
                experiences = experiences[1:]
                training_set = np.random.choice(experiences, batch_size, False)
                
                state_before = np.hstack([experience.state_before for experience in training_set])
                Q_values_before, H_before, Z1_before, Z2_before = network.forward(state_before)
                
                actions = np.hstack([experience.action for experience in training_set])
                Q_values_played = Q_values_before * actions
                
                states_after = np.hstack([experience.state_after for experience in training_set])
                Q_values_after, _, _, _ = network.forward(states_after)

                Q_values_after_played = np.max(Q_values_after, 0)
                
                rewards_played = np.hstack([experience.reward for experience in training_set])
                
                target = (rewards_played + gamma * Q_values_after_played) * actions
                network.descent(state_before, target, H_before, Q_values_played, Z1_before, Z2_before)
                
            ## THE EPISODE HAS ENDED, UPDATE...BE CAREFUL, THIS IS THE LAST STEP OF THE EPISODE
            if Done == 1:
                count.append(i)
                rewards.append(total_reward)
                break
                
            allowed_a = np.copy(allowed_a_next)

            i += 1  # UPDATE COUNTER FOR NUMBER OF ACTIONS
    return count, rewards

In [ ]:
## INITALISE YOUR NEURAL NETWORK...
network_replay = Network(N_h, N_in, N_a, eta=eta)

In [ ]:
train_and_test(replay, network_replay, "Experience Replay")

## Task 6a
We changed the administration of rewards as follows: Every regular action / move gets a slight punishment of -0.01, while a checkmate results in a reward of 1.0 and stale in a punishment of -1.0.


In [ ]:
env = Chess_Env(size_board, R=-0.01, R_draw=-1.0, R_checked=1.0)

## INITALISE YOUR NEURAL NETWORK...
network_simplified_a = Network(N_h, N_in, N_a, eta=eta)

In [ ]:
train_and_test(qlearn, network_simplified_a, "Q-Learning Simplified A")

## Task 6b
We removed the additional input in the state representation (number of possible moves for opponent king, information about checked-state).

In [ ]:
env = Chess_Env(size_board, extended_features=False)
_, X, _ = env.Initialise_game()

N_in_s = np.shape(X)[0]  ## INPUT SIZE

## INITALISE YOUR NEURAL NETWORK...
network_simplified_b = Network(N_h, N_in_s, N_a, eta=eta)

In [ ]:
train_and_test(qlearn, network_simplified_b, "Q-Learning Simplified B")

## Task 7
We implemented RMSProp as part of the Network class.

In [ ]:
env = Chess_Env(size_board)

## INITALISE YOUR NEURAL NETWORK...
network_qlearn_rmsprop = Network(N_h, N_in, N_a, eta=eta, rmsprop=True)

In [ ]:
train_and_test(qlearn, network_qlearn_rmsprop, "Q-Learning RMSprop")